<a href="https://colab.research.google.com/github/mricos/bird2vec/blob/master/bird2vec_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==1.12.0

In [2]:

!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo dpkg -i /content/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
print('add key: ok')
!sudo apt-get update
!sudo apt-get install cuda-libraries-9-0

--2019-04-23 03:25:10--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?jMGsV0EI2Q7_0m9NkoCxFjeCR22tyBbl88l67ZlGX0t3FaVyeYsJkMQbuTS6EglDntycVZCfQbeRP4qq6h7n_uwx6CBohGmFPgLxkqj7GXS_ag3kfQVmMPVbShs1eQZ6te3yXmtxG8fTp3d7XU38DTo6jm_Ezuudx93g1bWPM_RVYE3tXndq8jkTOtHdVgRxW9bO_xlsJpf_EqbuMUwz [following]
--2019-04-23 03:25:10--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?jMGsV0EI2Q7_0m9NkoCxFjeCR22tyBbl88l67ZlGX0t3FaVyeYsJkMQbuTS6EglDntycVZCfQbeRP4qq6h7n_uwx6CBohGmFPgLxkqj7GXS_ag

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#!pwd # just show pwd
#!mkdir train-data
#!cp "/content/gdrive/My Drive/wgtrain/model.ckpt-265.index" ./train-data/model.ckpt.index
#!cp "/content/gdrive/My Drive/wgtrain/model.ckpt-1801.data-00000-of-00001" ./train-data/model.ckpt.data-00000-of-00001
#!cp "/content/gdrive/My Drive/wgtrain/model.ckpt-265.meta" ./train-data/model.ckpt.meta
#!cp "/content/gdrive/My Drive/wgtrain/infer/infer.meta" ./train-data/infer.meta


In [0]:
# Load the model
import tensorflow as tf



In [9]:
!cat /content/gdrive/My\ Drive/wgtrain/checkpoint

model_checkpoint_path: "model.ckpt-1801"
all_model_checkpoint_paths: "model.ckpt-1713"
all_model_checkpoint_paths: "model.ckpt-1742"
all_model_checkpoint_paths: "model.ckpt-1772"
all_model_checkpoint_paths: "model.ckpt-1801"


In [18]:
path="/content/gdrive/My Drive/wgtrain/"
tf.reset_default_graph()
sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph(path+'infer/infer.meta')
graph = tf.get_default_graph()
saver.restore(sess, path+'model.ckpt-1801')

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/wgtrain/model.ckpt-1801


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
# Generate and display audio

# CHANGE THESE to change number of examples generated/displayed
ngenerate = 64
ndisplay = 4

import numpy as np
import PIL.Image
from IPython.display import display, Audio
import time as time

# Sample latent vectors
_z = (np.random.rand(ngenerate, 100) * 2.) - 1.

In [0]:
# Generate
z = graph.get_tensor_by_name('z:0')

In [0]:
G_z = graph.get_tensor_by_name('G_z:0')[:, :, 0]

In [0]:
G_z_spec = graph.get_tensor_by_name('G_z:0')

In [23]:
start = time.time()
_G_z, _G_z_spec = sess.run([G_z, G_z_spec], {z: _z})
print('Finished! (Took {} seconds)'.format(time.time() - start))

Finished! (Took 7.419282674789429 seconds)


In [24]:
print("z: Type: {} shape: {}".format(type(z),z.shape))
print("G_z: Type: {} shape: {}".format(type(G_z),G_z.shape))
print("G_z_spec: Type: {} shape: {}".format(type(G_z_spec),G_z_spec.shape))
print("_z: Type: {} shape: {}".format(type(z),_z.shape))
print("_G_z: Type: {} shape: {}".format(type(G_z),_G_z.shape))
print("_G_z_spec: Type: {} shape: {}".format(type(_G_z_spec),_G_z_spec.shape))
print("All 100 elements of first latent vector:\n{}".format(_z[0,:]))
print("All 100 elements of second latent vector:\n{}".format(_z[1,0:len(_z[0])]))

z: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (?, 100)
G_z: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (?, 16384)
G_z_spec: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (?, 16384, 1)
_z: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (64, 100)
_G_z: Type: <class 'tensorflow.python.framework.ops.Tensor'> shape: (64, 16384)
_G_z_spec: Type: <class 'numpy.ndarray'> shape: (64, 16384, 1)
All 100 elements of first latent vector:
[-0.89077328  0.10277784  0.00701718  0.89712186 -0.0970539  -0.61459273
 -0.5021706   0.54453757 -0.1385215   0.73492123  0.56814752  0.4411118
 -0.66729663 -0.0174943   0.0343202   0.63891204 -0.62202576 -0.30413446
  0.4228393  -0.74695729 -0.70106187  0.18640922 -0.89257194  0.50570816
  0.71930624 -0.22518917  0.1046916   0.50184773  0.41165915  0.01844834
 -0.74682196 -0.3550669  -0.08910885 -0.60436253 -0.13156795  0.38369187
 -0.95291614 -0.11967165 -0.14902532 -0.11686355 -0.9999091   0.6

In [26]:
for i in range(2):
  print('-' * 80)
  print('Example {}'.format(i))
  display(Audio(_G_z[i+20], rate=16000))

--------------------------------------------------------------------------------
Example 0


--------------------------------------------------------------------------------
Example 1
